<a href="https://colab.research.google.com/github/radf0001/NLP/blob/main/Pra%CC%81ctica_1_Extraccio%CC%81n_de_Informacio%CC%81n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 1 ( Extracción de Información)

## Profesora: Lisibonny Beato
### Período 3-2022-2023</div>


In [2]:
!python -m pip install textacy
!python -m pip install visualise_spacy_tree
!python -m spacy download en_core_web_trf
!python -m pip install spacy_transformers
!python  -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-27 16:40:47.903928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 16:40:50.483118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [79]:
# Importando algunas librerias

import pandas as pd
import re
import spacy
import textacy
import glob
import visualise_spacy_tree
from spacy.matcher import Matcher
import os
import sys
import en_core_web_trf
import spacy_transformers
import re
import nltk
import string
from nltk.corpus import stopwords

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>Para esta práctica estarán utilizando el corpus WNUT16, el cual encontrarán en la carpeta de dataset de nuestro curso. Para más información sobre este corpus ver el siguiente paper: https://aclanthology.org/W16-3919.pdf
<br />
<br />
Aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Nivel Básico
### Puntuación máxima de la tarea: 5 puntos
#### Limpieza y preparación de los datos, Extracción de palabras clave (KPE),  y reconocimiento de entidades relevantes (NER) con modelos heurísticos y pre-entrenados para  reconocer al menos 3 tipos de entidades.

In [5]:
# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

In [6]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

In [93]:
# Cargando Dataset
df = pd.DataFrame(columns=["Oracion"])
i = 0
oracion = ""
with open("/content/wnut16_test.txt") as bookfile:
    for line in bookfile:
        fields = line.split("\t")
        if(fields[0] != "\n"):
          if(oracion == ""):
            oracion += fields[0]
          else:
            oracion += " " + fields[0]
        else:
          df.loc[i, "Oracion"] = oracion;
          oracion = ""
          i += 1

In [ ]:
df

In [95]:
def limpiar(text):
    # Quitar numero de parrafo
    # text = re.sub('[0-9]', '', str(text))
    # Quitando algunos caracteres
    text = re.sub('\n', '', str(text))
    text = re.sub('\n ', '', str(text))
    text = re.sub('-', ' ', str(text))
    text = re.sub('- ', ' ', str(text))
    text = re.sub('"', '', str(text))
    text = re.sub("'s", '', str(text))

    return text

def longitud(text):
  return len(text.split())

In [96]:
df['Oracion_Limpia']=df['Oracion'].apply(limpiar)
df['Longitud']=df['Oracion_Limpia'].apply(longitud)


In [97]:
df

,Oracion,Oracion_Limpia,Longitud
0,New Orleans Mother 's Day Parade shooting . One of the people hurt was a 10-year-old girl . WHAT THE HELL IS WRONG WITH PEOPLE ?,New Orleans Mother Day Parade shooting . One of the people hurt was a 10 year old girl . WHAT THE HELL IS WRONG WITH PEOPLE ?,27
1,RT @hxranspizza : Going into school tomorrow like #KCA #Vote1DUK http://t.co/vvkoEEMjMX,RT @hxranspizza : Going into school tomorrow like #KCA #Vote1DUK http://t.co/vvkoEEMjMX,11
2,May e just a smile in your heart EILY CountdownBegins #PushAwardsLizQuens,May e just a smile in your heart EILY CountdownBegins #PushAwardsLizQuens,11
3,I could so do Thursday Club right now,I could so do Thursday Club right now,8
4,@therealdaftbear Albert Nobbs ( Glenn Close)is a woman living as a man in order to find work in the harsh environment of 19th-century Ireland,@therealdaftbear Albert Nobbs ( Glenn Close)is a woman living as a man in order to find work in the harsh environment of 19th century Ireland,25
...,...,...,...
3845,"Priest killed , another injured in US shooting : LOS ANGELES -- Police were hunting Thursday a gunman or gunmen who ... http://dlvr.it/5z2b10","Priest killed , another injured in US shooting : LOS ANGELES Police were hunting Thursday a gunman or gunmen who ... http://dlvr.it/5z2b10",22
3846,"Michael__Myerz : |LIVE NOW| Yes #meerkat https://t.co/XAaAUoyX0F January 03 , 2016 at 06:54 AM","Michael__Myerz : |LIVE NOW| Yes #meerkat https://t.co/XAaAUoyX0F January 03 , 2016 at 06:54 AM",14
3847,http://t.co/MoMmuSaDKE Daily Fantasy Basketball 2015 : Best NBA DFS DraftKings Lineup for March 22 #SportsTap http://t.co/eGuPiQ68r2,http://t.co/MoMmuSaDKE Daily Fantasy Basketball 2015 : Best NBA DFS DraftKings Lineup for March 22 #SportsTap http://t.co/eGuPiQ68r2,16
3848,@Toniakins no man alive has it all . But you can live enough in d moment knowing nothing in life is promised except tomorrow . Enjoy today,@Toniakins no man alive has it all . But you can live enough in d moment knowing nothing in life is promised except tomorrow . Enjoy today,27


In [98]:
count=0
for i in range(len(df)):
    if df.loc[i,'Longitud']==0:
        count+=1
print(count)

0


### Extraccion de KPES

In [99]:
# Vamos a tomar solo las primeras 4000 oraciones, ya que la funcion que convierte a documento de spacy no permite
# mas de 1000000 de caracteres para el documento.
oraciones_sample=df['Oracion_Limpia'][0:4000]

In [100]:
all_words = ' '.join([word for word in oraciones_sample])
all_words=all_words.lower()
all_words

"new orleans mother  day parade shooting . one of the people hurt was a 10 year old girl . what the hell is wrong with people ? rt @hxranspizza : going into school tomorrow like #kca #vote1duk http://t.co/vvkoeemjmx may e just a smile in your heart eily countdownbegins #pushawardslizquens i could so do thursday club right now @therealdaftbear albert nobbs ( glenn close)is a woman living as a man in order to find work in the harsh environment of 19th century ireland rt @1dasiacrew : louis ' interview with the sun about his own record label  78 productions limited  #proudoflouis http://t.co/hdswyedbio 9 injured in minneapolis nightclub shooting ; vikings player among victims : http://bring.mn/11865956 so happy moving into my new place on tuesday ! rt @aldenrichards02 : happy 10th weeksary sa ating lahat !!! #aldubourmissionislove #ikawyung tangi kung hiling sa 18th bday ko i drive by that motel almost every night . #mesashooting apple macbook pro a1278 13.3  laptop   md101ll/a ( june , 2

In [101]:
# Voy a convertir el string con todos los speeches a un formato de documento spacy
speeches=textacy.make_spacy_doc(all_words, lang='en_core_web_sm')

In [102]:
# Con este linea pedimos que se obtengan los mejores N KPEs desde el texto
textacy.extract.keyterms.textrank(speeches,topn=10)

[('rt @pojoktweet silakan rt manual tweet ini', 0.04372163913784202),
 ('rt rt @ezimuoh', 0.043097937605020724),
 ('new job tomorrow bruh rt @ohsaklapfriend', 0.025640009189868196),
 ('cest quand ya un mariage haitien que je mette un costard et je tape mon meilleur kompa rt @fiveshinki',
  0.025181972629560557),
 ('school tomorrow https://t.co/xspjwyl6tq rt @gatorsfb',
  0.024889844545386303),
 ('school tomorrow http://t.co/txuzonrssz rt @exoffical',
  0.024889844545386303),
 ('rt sa mga kinikilig jan wala pa man', 0.024866464894168862),
 ('school tomorrow rt @dailyrapfact', 0.02486128355118937),
 ('school tomorrow rt @angpoetnyo', 0.02486128355118937),
 ("c'est grave pas mon genre de photo xptdr jsuis insolente rt @comedypedia",
  0.024849277344643875)]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

## 2. Nivel Intermedio
### Puntuación máxima de la tarea: 2 puntos
#### Entrenar un modelo propio de NER mediante el pipeline de Spacy

## 3. Nivel Avanzado
### Puntuación máxima de la tarea: 3 puntos
#### Entrenar un modelo de NER mediante el uso de Conditional Random Fields. Recuerde ir más allá que donde llega el ejemplo al respecto provisto en clases.